In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import align.detect_face
import facenet
import glob
import cv2
import imutils
import numpy as np
import os
import tensorflow as tf

In [2]:
# Constants
minsize = 20
threshold = [0.6, 0.7, 0.7]
factor = 0.709
margin = 44
input_image_size = 160

In [3]:
print('Creating networks and loading parameters')
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, 'align')
sess = tf.Session()
with sess.as_default():
    pnet, rnet, onet = align.detect_face.create_mtcnn(sess, "align")
facenet.load_model("/home/neosai/Documents/model/facenet/models/20180408-102900/20180408-102900.pb")

Creating networks and loading parameters
Model filename: /home/neosai/Documents/model/facenet/models/20180408-102900/20180408-102900.pb


In [4]:
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]

In [5]:
def getFace(img):
    faces = []
    img_size = np.asarray(img.shape)[0:2]
    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    if not len(bounding_boxes) == 0:
        for face in bounding_boxes:
            if face[4] > 0.50:
                det = np.squeeze(face[0:4])
                bb = np.zeros(4, dtype=np.int32)
                bb[0] = np.maximum(det[0] - margin / 2, 0)
                bb[1] = np.maximum(det[1] - margin / 2, 0)
                bb[2] = np.minimum(det[2] + margin / 2, img_size[1])
                bb[3] = np.minimum(det[3] + margin / 2, img_size[0])
                cropped = img[bb[1]:bb[3], bb[0]:bb[2], :]
                resized = cv2.resize(cropped, (input_image_size,input_image_size),interpolation=cv2.INTER_CUBIC)
                prewhitened = facenet.prewhiten(resized)
                faces.append({'face':resized,'rect':[bb[0],bb[1],bb[2],bb[3]],'embedding':getEmbedding(prewhitened)})
    return faces

In [6]:
def getEmbedding(resized):
    reshaped = resized.reshape(-1,input_image_size,input_image_size,3)
    feed_dict = {images_placeholder: reshaped, phase_train_placeholder: False}
    embedding = sess.run(embeddings, feed_dict=feed_dict)
    return embedding

In [7]:
path = "../data/people_known/"
for image_name in glob.glob(path + "*"):
    print(image_name)
    img = cv2.imread(image_name)
    print(img)
    print(img.shape)
    face_encode = getFace(img)
    print(face_encode)

../data/people_known/Tong Thi Thuy.1.jpg
[[[ 78 192 178]
  [ 79 193 179]
  [ 77 194 179]
  ...
  [192 218 224]
  [192 218 224]
  [192 218 224]]

 [[ 81 194 180]
  [ 81 195 181]
  [ 82 196 182]
  ...
  [193 219 225]
  [193 219 225]
  [192 218 224]]

 [[ 84 195 181]
  [ 84 195 181]
  [ 85 196 182]
  ...
  [193 219 225]
  [193 219 225]
  [192 218 224]]

 ...

 [[225 240 255]
  [225 240 255]
  [224 239 255]
  ...
  [191 219 243]
  [191 219 243]
  [192 220 244]]

 [[223 238 254]
  [223 238 254]
  [222 237 253]
  ...
  [191 219 243]
  [192 220 244]
  [193 221 245]]

 [[222 237 253]
  [222 237 253]
  [222 237 253]
  ...
  [192 220 244]
  [192 220 244]
  [193 221 245]]]
(1284, 1061, 3)


NameError: name 'images_placeholder' is not defined